In [ ]:
import telebot
from langchain_groq import ChatGroq
import re

# Initialize the LLM model
llm = ChatGroq(
    temperature=0,
    groq_api_key="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",
    model_name="deepseek-r1-distill-llama-70b"
)

# Initialize the bot
bot = telebot.TeleBot('7774959017:AAEQZJGWOM-iLZzKg6Gs3JB8jgm4lWVkpF8')

# Command to start the bot
@bot.message_handler(commands=['start','hi','hello'])
def start(message):
    bot.send_message(message.chat.id, "Hi! I am Rupesh's personal assistant. How can i help you?", parse_mode='html')

# Handler for incoming text messages
@bot.message_handler(content_types='text')

def echo(message):
    # Get the response from the LLM based on user input
    response = llm.invoke(message.text)  # Send message.text to LLM and get response
    
    # Sanitize the response by removing unsupported HTML tags
    sanitized_response = re.sub(r'</?[^>]+>', '', response.content)  # Remove all HTML tags
    
    # Print the sanitized response (for debugging purposes)
    print(message.text)
    print(sanitized_response)
    
    # Send the sanitized response back to the user as the bot's reply
    bot.send_message(message.chat.id, sanitized_response, parse_mode='html')

@bot.message_handler(func=lambda message: True)
def handle_all_messages(message):
    # Your bot's response logic here
    bot.send_message(message.chat.id, "I received your message!")

# Run the bot
bot.infinity_polling()


2025-05-04 18:43:00,664 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


hi




Hello! How can I assist you today? 😊


2025-05-04 18:43:48,650 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


start




Hello! How can I assist you today? 😊


2025-05-04 18:47:16,740 (__init__.py:1121 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2025-05-04 18:47:16,740 - TeleBot - ERROR - Infinity polling: polling exited
2025-05-04 18:47:16,742 (__init__.py:1123 MainThread) ERROR - TeleBot: "Break infinity polling"
2025-05-04 18:47:16,742 - TeleBot - ERROR - Break infinity polling


In [45]:
import telebot

# Initialize the bot
bot = telebot.TeleBot('7774959017:AAEQZJGWOM-iLZzKg6Gs3JB8jgm4lWVkpF8')

# Command to start the bot
@bot.message_handler(commands=['start'])
def start(message):
    bot.send_message(message.chat.id, "Hi! Send me any file, and I’ll store it for you!")

# Handler for file uploads
@bot.message_handler(content_types=['document', 'photo', 'audio', 'video'])
def handle_files(message):
    try:
        # Get file ID
        if message.document:
            file_id = message.document.file_id
            file_name = message.document.file_name
        elif message.photo:
            file_id = message.photo[-1].file_id  # Get the largest resolution photo
            file_name = "photo.jpg"
        elif message.audio:
            file_id = message.audio.file_id
            file_name = message.audio.file_name
        elif message.video:
            file_id = message.video.file_id
            file_name = message.video.file_name
        else:
            bot.send_message(message.chat.id, "Unsupported file type.")
            return

        # Download the file
        file_info = bot.get_file(file_id)
        downloaded_file = bot.download_file(file_info.file_path)

        # Save the file locally
        with open(file_name, 'wb') as new_file:
            new_file.write(downloaded_file)

        bot.send_message(message.chat.id, f"File '{file_name}' has been saved successfully!")

    except Exception as e:
        bot.send_message(message.chat.id, "An error occurred while processing your file.")
        print(f"Error: {e}")

# Run the bot
bot.infinity_polling()


2025-05-04 18:54:33,376 (__init__.py:1121 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2025-05-04 18:54:33,376 - TeleBot - ERROR - Infinity polling: polling exited
2025-05-04 18:54:33,376 (__init__.py:1123 MainThread) ERROR - TeleBot: "Break infinity polling"
2025-05-04 18:54:33,376 - TeleBot - ERROR - Break infinity polling


In [46]:
import telebot
import os
import json

# Initialize the bot
BOT_TOKEN = '7774959017:AAEQZJGWOM-iLZzKg6Gs3JB8jgm4lWVkpF8'
bot = telebot.TeleBot(BOT_TOKEN)

# Directory to store uploaded files
UPLOAD_DIR = "uploaded_files"
os.makedirs(UPLOAD_DIR, exist_ok=True)

# File metadata storage
METADATA_FILE = "file_metadata.json"

# Load metadata or create a new one
if os.path.exists(METADATA_FILE):
    with open(METADATA_FILE, 'r') as f:
        file_metadata = json.load(f)
else:
    file_metadata = {}

# Command to start the bot
@bot.message_handler(commands=['start'])
def start(message):
    bot.send_message(
        message.chat.id,
        "Hello! I am your personal assistant for file sharing. You can:\n\n"
        "- Upload files by sending them to me.\n"
        "- Search for files by name.\n"
        "- Download your files.\n\n"
        "Commands:\n"
        "/search <file_name> - Search for a file\n"
        "Send me a file to upload!"
    )

# Handler for file uploads
@bot.message_handler(content_types=['document', 'photo', 'audio', 'video'])
def handle_files(message):
    try:
        # Determine file type and extract details
        if message.document:
            file_id = message.document.file_id
            file_name = message.document.file_name
        elif message.photo:
            file_id = message.photo[-1].file_id
            file_name = f"photo_{message.chat.id}_{message.message_id}.jpg"
        elif message.audio:
            file_id = message.audio.file_id
            file_name = message.audio.file_name
        elif message.video:
            file_id = message.video.file_id
            file_name = message.video.file_name
        else:
            bot.send_message(message.chat.id, "Unsupported file type.")
            return

        # Download the file
        file_info = bot.get_file(file_id)
        downloaded_file = bot.download_file(file_info.file_path)

        # Save the file locally
        file_path = os.path.join(UPLOAD_DIR, file_name)
        with open(file_path, 'wb') as new_file:
            new_file.write(downloaded_file)

        # Store metadata
        file_metadata[file_name] = {
            "owner_id": message.chat.id,
            "file_path": file_path,
            "file_id": file_id
        }
        with open(METADATA_FILE, 'w') as f:
            json.dump(file_metadata, f)

        bot.send_message(message.chat.id, f"File '{file_name}' uploaded successfully!")

    except Exception as e:
        bot.send_message(message.chat.id, "An error occurred while processing your file.")
        print(f"Error: {e}")

# Command to search for files
@bot.message_handler(commands=['search'])
def search_files(message):
    query = message.text.replace("/search", "").strip()
    if not query:
        bot.send_message(message.chat.id, "Please provide a file name to search for.")
        return

    # Search for matching files
    results = [
        name for name, meta in file_metadata.items()
        if query.lower() in name.lower() and meta['owner_id'] == message.chat.id
    ]

    if results:
        bot.send_message(message.chat.id, "Found the following files:")
        for result in results:
            bot.send_message(message.chat.id, result)
    else:
        bot.send_message(message.chat.id, "No matching files found.")

# Handler for file download requests
@bot.message_handler(content_types=['text'])
def send_file(message):
    file_name = message.text.strip()
    if file_name in file_metadata:
        meta = file_metadata[file_name]
        if meta['owner_id'] == message.chat.id:
            with open(meta['file_path'], 'rb') as file:
                bot.send_document(message.chat.id, file)
        else:
            bot.send_message(message.chat.id, "You are not authorized to access this file.")
    else:
        bot.send_message(message.chat.id, "File not found. Use /search to find available files.")

# Run the bot
bot.infinity_polling()


2025-05-04 18:59:10,200 (__init__.py:1121 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2025-05-04 18:59:10,200 - TeleBot - ERROR - Infinity polling: polling exited
2025-05-04 18:59:10,200 (__init__.py:1123 MainThread) ERROR - TeleBot: "Break infinity polling"
2025-05-04 18:59:10,200 - TeleBot - ERROR - Break infinity polling


In [47]:
import telebot
from langchain_groq import ChatGroq
import re

# Initialize the LLM model
llm = ChatGroq(
    temperature=0,
    groq_api_key="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",
    model_name="deepseek-r1-distill-llama-70b"
)

# Initialize the bot
bot = telebot.TeleBot('7774959017:AAEQZJGWOM-iLZzKg6Gs3JB8jgm4lWVkpF8')

# Command to start the bot
@bot.message_handler(commands=['start', 'hi', 'hello'])
def start(message):
    if message.chat.type == "private":
        bot.send_message(
            message.chat.id,
            "Hi! I am Rupesh's personal assistant. How can I help you?",
            parse_mode='html'
        )
    elif message.chat.type in ["group", "supergroup"]:
        bot.send_message(
            message.chat.id,
            "Hi everyone! I am Rupesh's assistant. Tag me (@YourBotUsername) if you need help!",
            parse_mode='html'
        )

# Handler for tagged messages in a group
@bot.message_handler(func=lambda message: message.chat.type in ["group", "supergroup"] and f"@{bot.get_me().username}" in message.text)
def handle_group_message(message):
    # Extract the message content after the bot's username
    user_message = message.text.replace(f"@{bot.get_me().username}", "").strip()

    # Get the response from the LLM
    response = llm.invoke(user_message)
    sanitized_response = re.sub(r'</?[^>]+>', '', response.content)  # Remove unsupported HTML tags

    # Send the response back to the group
    bot.send_message(message.chat.id, sanitized_response, parse_mode='html')

# Handler for private messages
@bot.message_handler(content_types='text')
def handle_private_message(message):
    if message.chat.type == "private":
        # Get the response from the LLM
        response = llm.invoke(message.text)
        sanitized_response = re.sub(r'</?[^>]+>', '', response.content)  # Remove unsupported HTML tags

        # Send the sanitized response back to the user
        bot.send_message(message.chat.id, sanitized_response, parse_mode='html')

# Fallback handler for unsupported messages
@bot.message_handler(func=lambda message: True)
def handle_all_messages(message):
    if message.chat.type == "private":
        bot.send_message(message.chat.id, "I received your message but couldn't process it.")
    elif message.chat.type in ["group", "supergroup"]:
        bot.send_message(message.chat.id, "I only respond to tagged messages. Tag me with @YourBotUsername.")

# Run the bot
bot.infinity_polling()


2025-05-04 19:08:50,616 (__init__.py:1115 MainThread) ERROR - TeleBot: "Infinity polling exception: HTTPSConnectionPool(host='api.telegram.org', port=443): Read timed out. (read timeout=25)"
2025-05-04 19:08:50,616 - TeleBot - ERROR - Infinity polling exception: HTTPSConnectionPool(host='api.telegram.org', port=443): Read timed out. (read timeout=25)
2025-05-04 19:08:50,625 (__init__.py:1117 MainThread) ERROR - TeleBot: "Exception traceback:
Traceback (most recent call last):
  File "c:\Users\dewan\anaconda3\Lib\site-packages\urllib3\connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "c:\Users\dewan\anaconda3\Lib\site-packages\urllib3\connection.py", line 507, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\dewan\anaconda3\Lib\http\client.py", line 1428, in getresponse
    response.begin()
  File "c:\Users\dewan\anaconda3\Lib\http\client.py", 

In [48]:
import telebot

# Replace with your bot's token
BOT_TOKEN = "7774959017:AAEQZJGWOM-iLZzKg6Gs3JB8jgm4lWVkpF8"

bot = telebot.TeleBot(BOT_TOKEN)

# Remove the webhook
bot.remove_webhook()

print("Webhook has been removed!")

Webhook has been removed!
